In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/preprocessed-raw-mat-csv/mat-csv-actual/mat-csv-actual/csv_files_from_mat2/02010030rest 20160324 1054..csv
/kaggle/input/preprocessed-raw-mat-csv/mat-csv-actual/mat-csv-actual/csv_files_from_mat2/02020025rest 20150713 1519..csv
/kaggle/input/preprocessed-raw-mat-csv/mat-csv-actual/mat-csv-actual/csv_files_from_mat2/02010013rest 20150703 1333..csv
/kaggle/input/preprocessed-raw-mat-csv/mat-csv-actual/mat-csv-actual/csv_files_from_mat2/02020016rest 20150701 1040..csv
/kaggle/input/preprocessed-raw-mat-csv/mat-csv-actual/mat-csv-actual/csv_files_from_mat2/02020015_rest 20150630 1527.csv
/kaggle/input/preprocessed-raw-mat-csv/mat-csv-actual/mat-csv-actual/csv_files_from_mat2/02010022restnew 20150724 14.csv
/kaggle/input/preprocessed-raw-mat-csv/mat-csv-actual/mat-csv-actual/csv_files_from_mat2/02020027rest 20150713 1049..csv
/kaggle/input/preprocessed-raw-mat-csv/mat-csv-actual/mat-csv-actual/csv_files_from_mat2/02010008_rest 20150619 1653.csv
/kaggle/input/preprocessed-raw-m

# **Setup, load & preprocessing, save splits**

In [8]:
# CELL 1: Setup + Load + Preprocess + Save splits
import os, re, math, json, warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import IncrementalPCA

# CONFIG
DATA_DIR    = '/kaggle/input/preprocessed-raw-mat-csv/mat-csv-actual/mat-csv-actual/csv_files_from_mat2'
OUTPUT_DIR  = '/kaggle/working/dl-results-3'
os.makedirs(OUTPUT_DIR, exist_ok=True)

SAMPLE_FRAC      = 1.0        # set 0.1 for quick tests
USE_IPCA         = True
IPCA_COMPONENTS  = 128
IPCA_BATCH       = 5000
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# GPU memory growth (optional)
gpus = tf.config.experimental.list_physical_devices('GPU')
for g in gpus:
    try:
        tf.config.experimental.set_memory_growth(g, True)
    except Exception:
        pass

# helpers
def detect_eeg_columns(columns):
    regex = re.compile(r'^(?:EEG[_\-\s]?|E[_\-\s]?)(0*?)(\d{1,3})$', flags=re.I)
    found = {}
    for c in columns:
        m = regex.match(c.strip())
        if m:
            num = int(m.group(2))
            if 1 <= num <= 128:
                found[num] = c
    if found:
        return [found[i] for i in sorted(found.keys())]
    # fallback
    return [c for c in columns if re.match(r'^(E|EEG)\d+', c, flags=re.I)]

def to_binary_label_series(s):
    s = s.dropna()
    if s.empty: return None
    s_num = pd.to_numeric(s, errors='coerce')
    if s_num.notna().all():
        uniq = set(np.unique(s_num))
        if uniq.issubset({0,1}): return s_num.astype(int)
        if uniq.issubset({1,2}): return s_num.map({1:0,2:1}).astype(int)
        med = float(s_num.median()); return (s_num > med).astype(int)
    s_str = s.astype(str)
    unique_vals = s_str.unique()
    if len(unique_vals) == 1: return s_str.map({unique_vals[0]:0}).astype(int)
    if len(unique_vals) == 2:
        le = LabelEncoder().fit(unique_vals)
        return pd.Series(le.transform(s_str), index=s_str.index).astype(int)
    mode_val = s_str.mode().iat[0]; return (s_str != mode_val).astype(int)

# 1) Read CSVs
csvs = sorted([f for f in os.listdir(DATA_DIR) if f.endswith('.csv')])
if len(csvs)==0:
    raise RuntimeError("No CSV files in DATA_DIR")
print("Found", len(csvs), "CSV files.")

parts = []
for fn in csvs:
    path = os.path.join(DATA_DIR, fn)
    df = pd.read_csv(path, engine='python')
    if SAMPLE_FRAC is not None and 0 < SAMPLE_FRAC < 1.0:
        df = df.sample(frac=SAMPLE_FRAC, random_state=SEED)
    df['__source_file'] = os.path.splitext(fn)[0]
    parts.append(df)
combined = pd.concat(parts, ignore_index=True)
print("Combined shape:", combined.shape)

# 2) label detection (prefer epoch, label, condition)
label_cols_try = ['epoch','label','condition','cond','target']
label_series = None
for c in label_cols_try:
    if c in combined.columns:
        s = to_binary_label_series(combined[c])
        if s is not None:
            label_series = pd.Series(index=combined.index, dtype=int)
            label_series.loc[combined[c].dropna().index] = s
            label_series = label_series.fillna(0).astype(int)
            print("Using", c, "as labels.")
            break
if label_series is None:
    # fallback search
    for c in combined.columns:
        if c.startswith('__'): continue
        s = to_binary_label_series(combined[c])
        if s is not None:
            label_series = pd.Series(index=combined.index, dtype=int)
            label_series.loc[combined[c].dropna().index] = s
            label_series = label_series.fillna(0).astype(int)
            print("Fallback using", c, "as labels.")
            break
if label_series is None:
    raise RuntimeError("No suitable label column found. Ensure 'epoch'/'label' exists.")

print("Label distribution:", label_series.value_counts().to_dict())
if label_series.nunique() <= 1:
    print("Detected single class after mapping — abort and inspect label columns.")
    raise RuntimeError("Single-class dataset. Fix labels.")

combined['__label'] = label_series.astype(int)

# 3) Detect EEG columns & form feature matrix
eeg_cols = detect_eeg_columns(combined.columns)
if not eeg_cols:
    raise RuntimeError("No EEG columns detected; check column names.")
print("Detected EEG columns:", len(eeg_cols))
# drop known metadata columns
drop_cols = {'time','condition','label','epoch','__source_file','__label'}
feature_cols = [c for c in eeg_cols if c not in drop_cols]
if len(feature_cols) == 0:
    raise RuntimeError("No feature columns after filtering.")
X_full = combined[feature_cols].to_numpy(dtype=np.float32)
y = combined['__label'].to_numpy(dtype=np.int32)
print("X_full shape:", X_full.shape, "y shape:", y.shape)

# impute NaNs
if np.isnan(X_full).any():
    col_means = np.nanmean(X_full, axis=0)
    inds = np.where(np.isnan(X_full)); X_full[inds] = np.take(col_means, inds[1])
    print("Imputed NaNs.")

# 4) Optional IncrementalPCA
if USE_IPCA and IPCA_COMPONENTS is not None and 0 < IPCA_COMPONENTS < X_full.shape[1]:
    print("Running IncrementalPCA...")
    ipca = IncrementalPCA(n_components=IPCA_COMPONENTS)
    n = X_full.shape[0]; bs = IPCA_BATCH
    for i in range(0, n, bs):
        ipca.partial_fit(X_full[i:i+bs])
    X_reduced = np.empty((n, IPCA_COMPONENTS), dtype=np.float32)
    for i in range(0, n, bs):
        X_reduced[i:i+bs] = ipca.transform(X_full[i:i+bs]).astype(np.float32)
    X = X_reduced
else:
    X = X_full
print("Post-PCA shape:", X.shape)

# 5) scale and split (save splits for model cells)
scaler = StandardScaler()
X = scaler.fit_transform(X).astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
# persist splits so model cells can load them
np.savez_compressed(os.path.join(OUTPUT_DIR, 'data_split.npz'),
                    X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)
print("Saved data_split.npz to", OUTPUT_DIR)
# create empty models_results.json if not exists
res_path = os.path.join(OUTPUT_DIR, 'models_results.json')
if not os.path.exists(res_path):
    with open(res_path,'w') as f: json.dump([], f)
print("Cell 1 done.")


Found 51 CSV files.
Combined shape: (3862388, 133)
Using epoch as labels.
Label distribution: {0: 1932951, 1: 1929437}
Detected EEG columns: 128
X_full shape: (3862388, 128) y shape: (3862388,)
Post-PCA shape: (3862388, 128)
Saved data_split.npz to /kaggle/working/dl-results-3
Cell 1 done.


# **Utility functions**

In [2]:
# CELL 2: Utility functions for model cells (run once)
import os, json, numpy as np
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score, accuracy_score

OUTPUT_DIR = '/kaggle/working/dl-results-3'
def load_data_splits():
    p = os.path.join(OUTPUT_DIR, 'data_split.npz')
    d = np.load(p)
    return d['X_train'], d['X_test'], d['y_train'], d['y_test']

def save_model_result(res):
    """Append JSON-serializable result dict to models_results.json"""
    p = os.path.join(OUTPUT_DIR, 'models_results.json')
    lst = []
    if os.path.exists(p):
        with open(p,'r') as f:
            try:
                lst = json.load(f)
            except Exception:
                lst = []
    lst.append(res)
    with open(p,'w') as f:
        json.dump(lst, f)

def make_result_dict(name, model, X_test, y_test, history=None):
    # predict probabilities where possible
    try:
        probs = model.predict(X_test, verbose=0).ravel()
    except Exception:
        # if model expects 3D or 4D, let caller reshape X_test appropriately before calling make_result_dict
        probs = model.predict(X_test, verbose=0).ravel()
    preds = (probs >= 0.5).astype(int)
    acc = float(accuracy_score(y_test, preds))
    try:
        roc_auc = float(roc_auc_score(y_test, probs))
    except Exception:
        roc_auc = None
    rep = classification_report(y_test, preds)
    cm = confusion_matrix(y_test, preds).tolist()
    try:
        fpr,tpr,_ = roc_curve(y_test, probs)
        fpr = fpr.tolist(); tpr = tpr.tolist()
    except Exception:
        fpr,tpr = [], []
    hist_dict = history.history if history is not None else {}
    # convert numpy types in hist to lists
    clean_hist = {k: (list(np.array(v).astype(float)) if hasattr(v,'__iter__') else v) for k,v in hist_dict.items()}
    res = {
        'name': name,
        'accuracy': acc,
        'roc_auc': roc_auc,
        'class_report': rep,
        'conf_mat': cm,
        'fpr': fpr,
        'tpr': tpr,
        'history': clean_hist
    }
    return res

print("Cell 2 loaded utilities.")

Cell 2 loaded utilities.


# **GAN-Fixed**

In [ ]:
# ================================
# CELL 3 — Vanilla GAN (Kaggle-optimized, NO early stopping)
# - saves ONLY best generator + best discriminator weights (.weights.h5)
# - runs ALL 1000 GAN epochs and ALL 50 classifier epochs
# - no synthetic datasets saved, minimal PNGs + JSON
# - OPTIMIZED: cached synthetic generation, tf.data for classifier
# - FIX: uses actual y_train labels for real data (not all 1s)
# ================================
import os, time, gc
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score

# -------- CONFIG --------
OUTPUT_DIR = "/kaggle/working/dl-results-3"
os.makedirs(OUTPUT_DIR, exist_ok=True)

LATENT_DIM = 100
EPOCHS_GAN = 1000          # run ALL 1000 epochs (no early stopping)
BATCH = 32
CLASSIFIER_EPOCHS = 50     # run ALL 50 epochs (no early stopping)
SEED = 42
SAVE_SIZE_CAP_BYTES = 2 * 1024**3   # 2 GiB safety cap (we'll warn if exceeded)
LOG_EVERY = 100
SYNTHETIC_CACHE_SIZE = None  # will be set to N_train (same as original code)

np.random.seed(SEED); tf.random.set_seed(SEED)

# -------- load data splits (Cell 1 saved .npz) --------
X_train, X_test, y_train, y_test = load_data_splits()
X = X_train.astype(np.float32)
FEATURES = X.shape[1]
N_train = X.shape[0]
SYNTHETIC_CACHE_SIZE = N_train  # match original: generate same amount as real data
print(f"[GAN] features={FEATURES} | n_train={N_train}")

# -------- build generator & discriminator (unchanged arch) --------
def build_generator():
    return tf.keras.Sequential([
        tf.keras.layers.Dense(256, input_dim=LATENT_DIM),
        tf.keras.layers.LeakyReLU(0.2),
        tf.keras.layers.Dense(512),
        tf.keras.layers.LeakyReLU(0.2),
        tf.keras.layers.Dense(1024),
        tf.keras.layers.LeakyReLU(0.2),
        tf.keras.layers.Dense(FEATURES, activation='tanh')
    ], name="generator")

def build_discriminator():
    return tf.keras.Sequential([
        tf.keras.layers.Dense(512, input_shape=(FEATURES,)),
        tf.keras.layers.LeakyReLU(0.2),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(256),
        tf.keras.layers.LeakyReLU(0.2),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ], name="discriminator")

generator = build_generator()
discriminator = build_discriminator()

opt_d = tf.keras.optimizers.Adam(0.0002, 0.5)
opt_g = tf.keras.optimizers.Adam(0.0002, 0.5)

# -------- prepare GAN model (compile with disc non-trainable for GAN-object) --------
discriminator.trainable = False
z = tf.keras.Input(shape=(LATENT_DIM,))
gan_out = discriminator(generator(z))
gan = tf.keras.Model(z, gan_out, name="gan_model")
gan.compile(optimizer=opt_g, loss="binary_crossentropy")

# Now compile discriminator separately for manual training loops
discriminator.trainable = True
discriminator.compile(optimizer=opt_d, loss="binary_crossentropy")

# -------- tf.data pipeline (fast + memory-safe) --------
AUTOTUNE = tf.data.AUTOTUNE
shuffle_buffer = min(10000, N_train)
ds = tf.data.Dataset.from_tensor_slices(X).shuffle(shuffle_buffer, seed=SEED).repeat().batch(BATCH).prefetch(AUTOTUNE)
ds_iter = iter(ds)

real_label = tf.ones((BATCH,1), tf.float32)
fake_label = tf.zeros((BATCH,1), tf.float32)

# -------- training step (tf.function for speed) --------
@tf.function
def train_step(real_batch):
    # discriminator on real
    with tf.GradientTape() as tape_d_real:
        pred_real = discriminator(real_batch, training=True)
        loss_real = tf.reduce_mean(tf.keras.losses.binary_crossentropy(real_label, pred_real))
    grads_real = tape_d_real.gradient(loss_real, discriminator.trainable_variables)
    opt_d.apply_gradients(zip(grads_real, discriminator.trainable_variables))

    # discriminator on fake
    noise = tf.random.normal((BATCH, LATENT_DIM))
    fake_batch = generator(noise, training=True)
    with tf.GradientTape() as tape_d_fake:
        pred_fake = discriminator(fake_batch, training=True)
        loss_fake = tf.reduce_mean(tf.keras.losses.binary_crossentropy(fake_label, pred_fake))
    grads_fake = tape_d_fake.gradient(loss_fake, discriminator.trainable_variables)
    opt_d.apply_gradients(zip(grads_fake, discriminator.trainable_variables))

    # generator step (try to fool discriminator) — compute grads manually
    noise2 = tf.random.normal((BATCH, LATENT_DIM))
    with tf.GradientTape() as tape_g:
        gen_out = generator(noise2, training=True)
        disc_out_for_g = discriminator(gen_out, training=False)   # freeze disc for generator update
        loss_g = tf.reduce_mean(tf.keras.losses.binary_crossentropy(real_label, disc_out_for_g))
    grads_g = tape_g.gradient(loss_g, generator.trainable_variables)
    opt_g.apply_gradients(zip(grads_g, generator.trainable_variables))

    return (loss_real + loss_fake) * 0.5, loss_g

# -------- adversarial training (NO early stopping - run all epochs) --------
print("[GAN] Adversarial training started (NO early stopping)...")
d_losses, g_losses = [], []
best_g_loss = np.inf
gen_best_path = os.path.join(OUTPUT_DIR, "generator_best.weights.h5")

start_time = time.time()
for epoch in range(EPOCHS_GAN):
    real_batch = next(ds_iter)
    d_val, g_val = train_step(real_batch)

    d_losses.append(float(d_val))
    g_losses.append(float(g_val))

    # save best generator weights (but don't stop early)
    cur_g = float(g_val)
    if cur_g < best_g_loss - 1e-8:
        best_g_loss = cur_g
        generator.save_weights(gen_best_path)   # tiny file

    # log progress
    if (epoch % LOG_EVERY) == 0 or epoch == EPOCHS_GAN-1:
        elapsed = time.time() - start_time
        print(f"[Epoch {epoch}/{EPOCHS_GAN}] D_loss={d_losses[-1]:.4f} | G_loss={g_losses[-1]:.4f} | best_g={best_g_loss:.4f} | elapsed={elapsed:.1f}s")

print(f"[GAN] Adversarial loop done in {time.time()-start_time:.1f}s. Best G loss: {best_g_loss:.5f}")
print("Generator best weights:", gen_best_path)

# -------- OPTIMIZED: pre-generate synthetic samples in bulk (avoid per-batch predict calls) --------
# FIX: Generate same amount as original (N_train samples) and use ACTUAL y_train labels
print(f"[Classifier] Pre-generating {SYNTHETIC_CACHE_SIZE} synthetic samples...")
noise_bulk = np.random.normal(0, 1, (SYNTHETIC_CACHE_SIZE, LATENT_DIM)).astype(np.float32)
synthetic_X = generator.predict(noise_bulk, batch_size=512, verbose=0)  # one bulk call
synthetic_y = np.zeros(SYNTHETIC_CACHE_SIZE, dtype=np.float32)          # synthetic = label 0

# FIX: use ACTUAL y_train labels for real data (not all 1s!)
# This matches original: combined_y = np.hstack((y, synthetic_labels))
combined_X = np.vstack([X, synthetic_X]).astype(np.float32)
combined_y = np.hstack([y_train.astype(np.float32), synthetic_y])  # ← KEY FIX

# shuffle combined data
perm = np.random.permutation(len(combined_X))
combined_X = combined_X[perm]
combined_y = combined_y[perm].reshape(-1, 1)

print(f"[Classifier] Combined training set: {combined_X.shape[0]} samples (real + synthetic)")
print(f"[Classifier] Label distribution: 0={np.sum(combined_y==0)}, 1={np.sum(combined_y==1)}")

# -------- tf.data pipeline for classifier (faster than Python generator) --------
train_ds = tf.data.Dataset.from_tensor_slices((combined_X, combined_y))
train_ds = train_ds.shuffle(buffer_size=min(50000, len(combined_X)), seed=SEED)
train_ds = train_ds.batch(BATCH).prefetch(AUTOTUNE)

# validation data - use actual test labels
val_ds = tf.data.Dataset.from_tensor_slices(
    (X_test.astype(np.float32), y_test.reshape(-1,1).astype(np.float32))
).batch(BATCH).prefetch(AUTOTUNE)

disc_best_path = os.path.join(OUTPUT_DIR, "discriminator_best.weights.h5")
# ModelCheckpoint saves best weights only (no early stopping)
checkpoint = ModelCheckpoint(disc_best_path, monitor="val_accuracy",
                             save_best_only=True, save_weights_only=True, verbose=1)

# Rebuild discriminator fresh for classifier training (avoid any leftover state)
discriminator = build_discriminator()
discriminator.compile(optimizer=tf.keras.optimizers.Adam(0.0002, 0.5),
                      loss="binary_crossentropy", metrics=["accuracy"])

print("[Classifier] Fine-tuning discriminator (NO early stopping - all 50 epochs)...")
history = discriminator.fit(
    train_ds,
    epochs=CLASSIFIER_EPOCHS,
    validation_data=val_ds,
    callbacks=[checkpoint],  # only checkpoint, no early stopping
    verbose=1
)

# Load best weights
discriminator.load_weights(disc_best_path)
print("Discriminator best weights loaded:", disc_best_path)

# -------- free memory from synthetic cache --------
del synthetic_X, combined_X, combined_y, noise_bulk
gc.collect()

# -------- evaluation & small PNGs (ROC + train curves) --------
y_prob = discriminator.predict(X_test, verbose=0).ravel()
y_pred = (y_prob >= 0.5).astype(int)

acc = float(np.mean(y_pred == y_test))
print(f"\n=== Accuracy: {acc:.4f} ===")
print("\n=== Classification Report (Discriminator) ===")
print(classification_report(y_test, y_pred))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

# ROC
try:
    auc_val = roc_auc_score(y_test, y_prob)
    print(f"ROC-AUC: {auc_val:.4f}")
except Exception:
    auc_val = None
fpr, tpr, _ = roc_curve(y_test, y_prob)

# plot ROC curve
plt.figure(figsize=(5,5))
plt.plot(fpr, tpr, color='blue', lw=2, label=f"AUC={auc_val:.3f}" if auc_val else "ROC")
plt.plot([0,1],[0,1],'k--', alpha=0.3)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Vanilla GAN Discriminator ROC")
plt.legend(loc="lower right")
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "gan_discriminator_roc.png"), dpi=100)
plt.show()
plt.close()

# Train/val accuracy curves
if history is not None:
    fig, axes = plt.subplots(1, 2, figsize=(10, 4))
    
    # accuracy
    axes[0].plot(history.history.get("accuracy", []), label="train_acc")
    axes[0].plot(history.history.get("val_accuracy", []), label="val_acc")
    axes[0].set_xlabel("Epoch")
    axes[0].set_ylabel("Accuracy")
    axes[0].set_title("Vanilla GAN - Accuracy")
    axes[0].legend()
    
    # loss
    axes[1].plot(history.history.get("loss", []), label="train_loss")
    axes[1].plot(history.history.get("val_loss", []), label="val_loss")
    axes[1].set_xlabel("Epoch")
    axes[1].set_ylabel("Loss")
    axes[1].set_title("Vanilla GAN - Loss")
    axes[1].legend()
    
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, "gan_train_val_curves.png"), dpi=100)
    plt.show()
    plt.close()

# -------- save JSON result entry (models_results.json via helper) --------
res = make_result_dict("Vanilla_GAN", discriminator, X_test, y_test, history)
res.update({
    "best_generator_weights": os.path.basename(gen_best_path),
    "best_discriminator_weights": os.path.basename(disc_best_path),
    "gan_epochs_ran": int(len(g_losses)),
    "d_losses": d_losses[-10:],  # save only last 10 for JSON size
    "g_losses": g_losses[-10:],
    "timestamp": time.time()
})
save_model_result(res)
print("[JSON] result appended to models_results.json")

# -------- storage guard: ensure saved files stay small (warn if > 2 GiB) --------
def human_mb(n): return f"{n/1024**2:.2f} MB"
total_bytes = 0
for root, _, files in os.walk(OUTPUT_DIR):
    for f in files:
        total_bytes += os.path.getsize(os.path.join(root, f))
if total_bytes > SAVE_SIZE_CAP_BYTES:
    print("⚠️ WARNING: OUTPUT_DIR size > 2 GiB — consider removing large artifacts.")
print(f"Output directory size: {total_bytes/1024**2:.2f} MB")

# -------- cleanup --------
K.clear_session()
gc.collect()

print("✅ CELL 3 finished — only BEST weights saved (generator + discriminator) + PNGs + JSON.")

[GAN] features=128 | n_train=3089910


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1764775763.852854    2551 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1764775763.853475    2551 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


[GAN] Adversarial training started (NO early stopping)...
[Epoch 0/1000] D_loss=0.7219 | G_loss=0.7106 | best_g=0.7106 | elapsed=4.0s
[Epoch 100/1000] D_loss=0.6071 | G_loss=0.8265 | best_g=0.5312 | elapsed=4.7s
[Epoch 200/1000] D_loss=0.6482 | G_loss=0.9554 | best_g=0.5312 | elapsed=5.3s
[Epoch 300/1000] D_loss=0.7745 | G_loss=0.7830 | best_g=0.5312 | elapsed=5.8s
[Epoch 400/1000] D_loss=0.6837 | G_loss=0.8747 | best_g=0.5312 | elapsed=6.4s
[Epoch 500/1000] D_loss=0.5650 | G_loss=0.9658 | best_g=0.5312 | elapsed=7.0s
[Epoch 600/1000] D_loss=0.6833 | G_loss=0.9006 | best_g=0.5312 | elapsed=7.5s
[Epoch 700/1000] D_loss=0.7142 | G_loss=0.7888 | best_g=0.5312 | elapsed=8.1s
[Epoch 800/1000] D_loss=0.6463 | G_loss=0.7163 | best_g=0.5312 | elapsed=8.7s
[Epoch 900/1000] D_loss=1.0218 | G_loss=0.7003 | best_g=0.5312 | elapsed=9.2s
[Epoch 999/1000] D_loss=0.6199 | G_loss=1.0684 | best_g=0.5312 | elapsed=9.8s
[GAN] Adversarial loop done in 9.8s. Best G loss: 0.53120
Generator best weights: /kag

I0000 00:00:1764775793.762480    2591 service.cc:148] XLA service 0x7c6a84007170 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1764775793.763328    2591 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1764775793.763349    2591 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1764775793.848361    2591 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1764775794.233780    2591 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


[Classifier] Combined training set: 6179820 samples (real + synthetic)
[Classifier] Label distribution: 0=4636271, 1=1543549
[Classifier] Fine-tuning discriminator (NO early stopping - all 50 epochs)...
Epoch 1/50
193120/193120 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7930 - loss: 0.3279
Epoch 1: val_accuracy improved from -inf to 0.68134, saving model to /kaggle/working/dl-results-3/discriminator_best.weights.h5
193120/193120 ━━━━━━━━━━━━━━━━━━━━ 465s 2ms/step - accuracy: 0.7930 - loss: 0.3279 - val_accuracy: 0.6813 - val_loss: 0.5616
Epoch 2/50
193120/193120 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8315 - loss: 0.2897
Epoch 2: val_accuracy improved from 0.68134 to 0.71285, saving model to /kaggle/working/dl-results-3/discriminator_best.weights.h5
193120/193120 ━━━━━━━━━━━━━━━━━━━━ 469s 2ms/step - accuracy: 0.8315 - loss: 0.2897 - val_accuracy: 0.7129 - val_loss: 0.5225
Epoch 3/50
193116/193120 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8410 - loss: 0.2786
Epoch 3: val